## Final Project - Group 3 - Credit Card Fraud Detection
Team: Sean Ely, Xiwang Li, Pedram, Amir, Rox, Arash

### Kaggle - Credit Card Fraud Detection
https://www.kaggle.com/mlg-ulb/creditcardfraud
#### Context
It is important that credit card companies are able to recognize fraudulent credit card transactions so that customers are not charged for items that they did not purchase.
#### Content
The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.
#### Inspiration
Identify fraudulent credit card transactions.

Given the class imbalance ratio, we recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful for unbalanced classification.
#### Acknowledgements
The dataset has been collected and analysed during a research collaboration of Worldline and the Machine Learning Group (http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection. More details on current and past projects on related topics are available on http://mlg.ulb.ac.be/BruFence and http://mlg.ulb.ac.be/ARTML

Please cite: Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015

In [2]:
%sh
#need to run ***ONCE*** to install SMOTE package
/home/ubuntu/databricks/python/bin/pip install 'imbalanced-learn<0.2.1'
pip freeze | grep imbalanced-learn

Requirement already satisfied: imbalanced-learn<0.2.1 in /databricks/python2/lib/python2.7/site-packages (0.2.0)
Requirement already satisfied: scipy>=0.17.0 in /databricks/python2/lib/python2.7/site-packages (from imbalanced-learn<0.2.1) (0.18.1)
Requirement already satisfied: numpy>=1.10.4 in /databricks/python2/lib/python2.7/site-packages (from imbalanced-learn<0.2.1) (1.11.1)
Requirement already satisfied: scikit-learn>=0.17.1 in /databricks/python2/lib/python2.7/site-packages (from imbalanced-learn<0.2.1) (0.18.1)
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 8.1.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [3]:
# File location and type
file_location = "/FileStore/tables/creditcard.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942518,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,0.0907941719789316,-0.551599533260813,-0.617800855762348,-0.991389847235408,-0.311169353699879,1.46817697209427,-0.470400525259478,0.207971241929242,0.0257905801985591,0.403992960255733,0.251412098239705,-0.018306777944153,0.277837575558899,-0.110473910188767,0.0669280749146731,0.128539358273528,-0.189114843888824,0.133558376740387,-0.0210530534538215,149.62,0
0,1.19185711131486,0.26615071205963,0.16648011335321,0.448154078460911,0.0600176492822243,-0.0823608088155687,-0.0788029833323113,0.0851016549148104,-0.255425128109186,-0.166974414004614,1.61272666105479,1.06523531137287,0.48909501589608,-0.143772296441519,0.635558093258208,0.463917041022171,-0.114804663102346,-0.183361270123994,-0.145783041325259,-0.0690831352230203,-0.225775248033138,-0.638671952771851,0.101288021253234,-0.339846475529127,0.167170404418143,0.125894532368176,-0.00898309914322813,0.0147241691924927,2.69,0
1,-1.35835406159823,-1.34016307473609,1.77320934263119,0.379779593034328,-0.503198133318193,1.80049938079263,0.791460956450422,0.247675786588991,-1.51465432260583,0.207642865216696,0.624501459424895,0.066083685268831,0.717292731410831,-0.165945922763554,2.34586494901581,-2.89008319444231,1.10996937869599,-0.121359313195888,-2.26185709530414,0.524979725224404,0.247998153469754,0.771679401917229,0.909412262347719,-0.689280956490685,-0.327641833735251,-0.139096571514147,-0.0553527940384261,-0.0597518405929204,378.66,0
1,-0.966271711572087,-0.185226008082898,1.79299333957872,-0.863291275036453,-0.0103088796030823,1.24720316752486,0.23760893977178,0.377435874652262,-1.38702406270197,-0.0549519224713749,-0.226487263835401,0.178228225877303,0.507756869957169,-0.28792374549456,-0.631418117709045,-1.0596472454325,-0.684092786345479,1.96577500349538,-1.2326219700892,-0.208037781160366,-0.108300452035545,0.00527359678253453,-0.190320518742841,-1.17557533186321,0.647376034602038,-0.221928844458407,0.0627228487293033,0.0614576285006353,123.5,0
2,-1.15823309349523,0.877736754848451,1.548717846511,0.403033933955121,-0.407193377311653,0.0959214624684256,0.592940745385545,-0.270532677192282,0.817739308235294,0.753074431976354,-0.822842877946363,0.53819555014995,1.3458515932154,-1.11966983471731,0.175121130008994,-0.451449182813529,-0.237033239362776,-0.0381947870352842,0.803486924960175,0.408542360392758,-0.00943069713232919,0.79827849458971,-0.137458079619063,0.141266983824769,-0.206009587619756,0.502292224181569,0.219422229513348,0.215153147499206,69.99,0
2,-0.425965884412454,0.960523044882985,1.14110934232219,-0.168252079760302,0.42098688077219,-0.0297275516639742,0.476200948720027,0.260314333074874,-0.56867137571251,-0.371407196834471,1.34126198001957,0.359893837038039,-0.358090652573631,-0.137133700217612,0.517616806555742,0.401725895589603,-0.0581328233640131,0.0686531494425432,-0.0331937877876282,0.0849676720682049,-0.208253514656728,-0.559824796253248,-0.0263976679795373,-0.371426583174346,-0.232793816737034,0.105914779097957,0.253844224739337,0.0810802569229443,3.67,0
4,1.22965763450793,0.141003507049326,0.0453707735899449,1.20261273673594,0.191880988597645,0.272708122899098,-0.00515900288250983,0.0812129398830894,0.464959994783886,-0.0992543211289237,-1.41690724314928,-0.153825826253651,-0.75106271556262,0.16737196252175,0.0501435942254188,-0.443586797916727,0.00282051247234708,-0.61198733994012,-0.0455750446637976,-0.21963255278686,-0.167716265815783,-0.270709726172363,-0.154103786809305,-0.780055415004671,0.75013693580659,-0.257236845917139,0.0345074297438413,0.00516776890624916,4.99,0
7,-0.644269442348146,1.41796354547385,1.0743803763556,-0.492199018495015,0.948934094764157,0.428118462833089,1.12063135838353,-3.80786423873589,0.615374730667027,1.24937617815176,-0.6194677961219

In [4]:
# Imported Libraries
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [5]:
df.printSchema()

root
-- Time: decimal(10,0) (nullable = true)
-- V1: double (nullable = true)
-- V2: double (nullable = true)
-- V3: double (nullable = true)
-- V4: double (nullable = true)
-- V5: double (nullable = true)
-- V6: double (nullable = true)
-- V7: double (nullable = true)
-- V8: double (nullable = true)
-- V9: double (nullable = true)
-- V10: double (nullable = true)
-- V11: double (nullable = true)
-- V12: double (nullable = true)
-- V13: double (nullable = true)
-- V14: double (nullable = true)
-- V15: double (nullable = true)
-- V16: double (nullable = true)
-- V17: double (nullable = true)
-- V18: double (nullable = true)
-- V19: double (nullable = true)
-- V20: double (nullable = true)
-- V21: double (nullable = true)
-- V22: double (nullable = true)
-- V23: double (nullable = true)
-- V24: double (nullable = true)
-- V25: double (nullable = true)
-- V26: double (nullable = true)
-- V27: double (nullable = true)
-- V28: double (nullable = true)
-- Amount: double (nullable = true)
-- Class: integer (nullable = true)

In [6]:
df.describe('Amount').show()

+-------+------------------+
summary| Amount|
+-------+------------------+
 count| 284807|
 mean| 88.34961925093698|
 stddev|250.12010924018836|
 min| 0.0|
 max| 25691.16|
+-------+------------------+

In [7]:
# number of null or nan values
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+
Time| V1| V2| V3| V4| V5| V6| V7| V8| V9|V10|V11|V12|V13|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|V26|V27|V28|Amount|Class|
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+

In [8]:
# The classes are heavily skewed we need to solve this issue later.
print 'Number of frauds: ', df.filter(F.col('Class') == 1).count(), ", ", round((df.filter(F.col('Class') == 1).count())/float(df.count()) * 100, 3), '% of the dataset'
print 'Number of records', df.count()

Number of frauds: 492 , 0.173 % of the dataset
Number of records 284807

In [9]:
display(df.groupBy('class').count())

class,count
1,492
0,284315


In [10]:
display(df)

Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942518,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,0.0907941719789316,-0.551599533260813,-0.617800855762348,-0.991389847235408,-0.311169353699879,1.46817697209427,-0.470400525259478,0.207971241929242,0.0257905801985591,0.403992960255733,0.251412098239705,-0.018306777944153,0.277837575558899,-0.110473910188767,0.0669280749146731,0.128539358273528,-0.189114843888824,0.133558376740387,-0.0210530534538215,149.62,0
0,1.19185711131486,0.26615071205963,0.16648011335321,0.448154078460911,0.0600176492822243,-0.0823608088155687,-0.0788029833323113,0.0851016549148104,-0.255425128109186,-0.166974414004614,1.61272666105479,1.06523531137287,0.48909501589608,-0.143772296441519,0.635558093258208,0.463917041022171,-0.114804663102346,-0.183361270123994,-0.145783041325259,-0.0690831352230203,-0.225775248033138,-0.638671952771851,0.101288021253234,-0.339846475529127,0.167170404418143,0.125894532368176,-0.00898309914322813,0.0147241691924927,2.69,0
1,-1.35835406159823,-1.34016307473609,1.77320934263119,0.379779593034328,-0.503198133318193,1.80049938079263,0.791460956450422,0.247675786588991,-1.51465432260583,0.207642865216696,0.624501459424895,0.066083685268831,0.717292731410831,-0.165945922763554,2.34586494901581,-2.89008319444231,1.10996937869599,-0.121359313195888,-2.26185709530414,0.524979725224404,0.247998153469754,0.771679401917229,0.909412262347719,-0.689280956490685,-0.327641833735251,-0.139096571514147,-0.0553527940384261,-0.0597518405929204,378.66,0
1,-0.966271711572087,-0.185226008082898,1.79299333957872,-0.863291275036453,-0.0103088796030823,1.24720316752486,0.23760893977178,0.377435874652262,-1.38702406270197,-0.0549519224713749,-0.226487263835401,0.178228225877303,0.507756869957169,-0.28792374549456,-0.631418117709045,-1.0596472454325,-0.684092786345479,1.96577500349538,-1.2326219700892,-0.208037781160366,-0.108300452035545,0.00527359678253453,-0.190320518742841,-1.17557533186321,0.647376034602038,-0.221928844458407,0.0627228487293033,0.0614576285006353,123.5,0
2,-1.15823309349523,0.877736754848451,1.548717846511,0.403033933955121,-0.407193377311653,0.0959214624684256,0.592940745385545,-0.270532677192282,0.817739308235294,0.753074431976354,-0.822842877946363,0.53819555014995,1.3458515932154,-1.11966983471731,0.175121130008994,-0.451449182813529,-0.237033239362776,-0.0381947870352842,0.803486924960175,0.408542360392758,-0.00943069713232919,0.79827849458971,-0.137458079619063,0.141266983824769,-0.206009587619756,0.502292224181569,0.219422229513348,0.215153147499206,69.99,0
2,-0.425965884412454,0.960523044882985,1.14110934232219,-0.168252079760302,0.42098688077219,-0.0297275516639742,0.476200948720027,0.260314333074874,-0.56867137571251,-0.371407196834471,1.34126198001957,0.359893837038039,-0.358090652573631,-0.137133700217612,0.517616806555742,0.401725895589603,-0.0581328233640131,0.0686531494425432,-0.0331937877876282,0.0849676720682049,-0.208253514656728,-0.559824796253248,-0.0263976679795373,-0.371426583174346,-0.232793816737034,0.105914779097957,0.253844224739337,0.0810802569229443,3.67,0
4,1.22965763450793,0.141003507049326,0.0453707735899449,1.20261273673594,0.191880988597645,0.272708122899098,-0.00515900288250983,0.0812129398830894,0.464959994783886,-0.0992543211289237,-1.41690724314928,-0.153825826253651,-0.75106271556262,0.16737196252175,0.0501435942254188,-0.443586797916727,0.00282051247234708,-0.61198733994012,-0.0455750446637976,-0.21963255278686,-0.167716265815783,-0.270709726172363,-0.154103786809305,-0.780055415004671,0.75013693580659,-0.257236845917139,0.0345074297438413,0.00516776890624916,4.99,0
7,-0.644269442348146,1.41796354547385,1.0743803763556,-0.492199018495015,0.948934094764157,0.428118462833089,1.12063135838353,-3.80786423873589,0.615374730667027,1.24937617815176,-0.6194677961219

In [11]:
display(df)

Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942518,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,0.0907941719789316,-0.551599533260813,-0.617800855762348,-0.991389847235408,-0.311169353699879,1.46817697209427,-0.470400525259478,0.207971241929242,0.0257905801985591,0.403992960255733,0.251412098239705,-0.018306777944153,0.277837575558899,-0.110473910188767,0.0669280749146731,0.128539358273528,-0.189114843888824,0.133558376740387,-0.0210530534538215,149.62,0
0,1.19185711131486,0.26615071205963,0.16648011335321,0.448154078460911,0.0600176492822243,-0.0823608088155687,-0.0788029833323113,0.0851016549148104,-0.255425128109186,-0.166974414004614,1.61272666105479,1.06523531137287,0.48909501589608,-0.143772296441519,0.635558093258208,0.463917041022171,-0.114804663102346,-0.183361270123994,-0.145783041325259,-0.0690831352230203,-0.225775248033138,-0.638671952771851,0.101288021253234,-0.339846475529127,0.167170404418143,0.125894532368176,-0.00898309914322813,0.0147241691924927,2.69,0
1,-1.35835406159823,-1.34016307473609,1.77320934263119,0.379779593034328,-0.503198133318193,1.80049938079263,0.791460956450422,0.247675786588991,-1.51465432260583,0.207642865216696,0.624501459424895,0.066083685268831,0.717292731410831,-0.165945922763554,2.34586494901581,-2.89008319444231,1.10996937869599,-0.121359313195888,-2.26185709530414,0.524979725224404,0.247998153469754,0.771679401917229,0.909412262347719,-0.689280956490685,-0.327641833735251,-0.139096571514147,-0.0553527940384261,-0.0597518405929204,378.66,0
1,-0.966271711572087,-0.185226008082898,1.79299333957872,-0.863291275036453,-0.0103088796030823,1.24720316752486,0.23760893977178,0.377435874652262,-1.38702406270197,-0.0549519224713749,-0.226487263835401,0.178228225877303,0.507756869957169,-0.28792374549456,-0.631418117709045,-1.0596472454325,-0.684092786345479,1.96577500349538,-1.2326219700892,-0.208037781160366,-0.108300452035545,0.00527359678253453,-0.190320518742841,-1.17557533186321,0.647376034602038,-0.221928844458407,0.0627228487293033,0.0614576285006353,123.5,0
2,-1.15823309349523,0.877736754848451,1.548717846511,0.403033933955121,-0.407193377311653,0.0959214624684256,0.592940745385545,-0.270532677192282,0.817739308235294,0.753074431976354,-0.822842877946363,0.53819555014995,1.3458515932154,-1.11966983471731,0.175121130008994,-0.451449182813529,-0.237033239362776,-0.0381947870352842,0.803486924960175,0.408542360392758,-0.00943069713232919,0.79827849458971,-0.137458079619063,0.141266983824769,-0.206009587619756,0.502292224181569,0.219422229513348,0.215153147499206,69.99,0
2,-0.425965884412454,0.960523044882985,1.14110934232219,-0.168252079760302,0.42098688077219,-0.0297275516639742,0.476200948720027,0.260314333074874,-0.56867137571251,-0.371407196834471,1.34126198001957,0.359893837038039,-0.358090652573631,-0.137133700217612,0.517616806555742,0.401725895589603,-0.0581328233640131,0.0686531494425432,-0.0331937877876282,0.0849676720682049,-0.208253514656728,-0.559824796253248,-0.0263976679795373,-0.371426583174346,-0.232793816737034,0.105914779097957,0.253844224739337,0.0810802569229443,3.67,0
4,1.22965763450793,0.141003507049326,0.0453707735899449,1.20261273673594,0.191880988597645,0.272708122899098,-0.00515900288250983,0.0812129398830894,0.464959994783886,-0.0992543211289237,-1.41690724314928,-0.153825826253651,-0.75106271556262,0.16737196252175,0.0501435942254188,-0.443586797916727,0.00282051247234708,-0.61198733994012,-0.0455750446637976,-0.21963255278686,-0.167716265815783,-0.270709726172363,-0.154103786809305,-0.780055415004671,0.75013693580659,-0.257236845917139,0.0345074297438413,0.00516776890624916,4.99,0
7,-0.644269442348146,1.41796354547385,1.0743803763556,-0.492199018495015,0.948934094764157,0.428118462833089,1.12063135838353,-3.80786423873589,0.615374730667027,1.24937617815176,-0.6194677961219

In [12]:
df_fraud = df.filter(F.col('Class') == 1)
df_no_fraud_sample = df.filter(F.col('Class') == 0).sample(False, 0.1).limit(4920) #downsaple no fraud class 

df_sample = df_fraud.union(df_no_fraud_sample)
display(df_sample)

Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
406,-2.3122265423263,1.95199201064158,-1.60985073229769,3.9979055875468,-0.522187864667764,-1.42654531920595,-2.53738730624579,1.39165724829804,-2.77008927719433,-2.77227214465915,3.20203320709635,-2.89990738849473,-0.595221881324605,-4.28925378244217,0.389724120274487,-1.14074717980657,-2.83005567450437,-0.0168224681808257,0.416955705037907,0.126910559061474,0.517232370861764,-0.0350493686052974,-0.465211076182388,0.320198198514526,0.0445191674731724,0.177839798284401,0.261145002567677,-0.143275874698919,0.0,1
472,-3.0435406239976,-3.15730712090228,1.08846277997285,2.2886436183814,1.35980512966107,-1.06482252298131,0.325574266158614,-0.0677936531906277,-0.270952836226548,-0.838586564582682,-0.414575448285725,-0.503140859566824,0.676501544635863,-1.69202893305906,2.00063483909015,0.666779695901966,0.599717413841732,1.72532100745514,0.283344830149495,2.10233879259444,0.661695924845707,0.435477208966341,1.37596574254306,-0.293803152734021,0.279798031841214,-0.145361714815161,-0.252773122530705,0.0357642251788156,529.0,1
4462,-2.30334956758553,1.759247460267,-0.359744743330052,2.33024305053917,-0.821628328375422,-0.0757875706194599,0.562319782266954,-0.399146578487216,-0.238253367661746,-1.52541162656194,2.03291215755072,-6.56012429505962,0.0229373234890961,-1.47010153611197,-0.698826068579047,-2.28219382856251,-4.78183085597533,-2.61566494476124,-1.33444106667307,-0.430021867171611,-0.294166317554753,-0.932391057274991,0.172726295799422,-0.0873295379700724,-0.156114264651172,-0.542627889040196,0.0395659889264757,-0.153028796529788,239.93,1
6986,-4.39797444171999,1.35836702839758,-2.5928442182573,2.67978696694832,-1.12813094208956,-1.70653638774951,-3.49619729302467,-0.248777743025673,-0.24776789948008,-4.80163740602813,4.89584422347523,-10.9128193194019,0.184371685834387,-6.77109672468083,-0.00732618257771211,-7.35808322132346,-12.5984185405511,-5.13154862842983,0.308333945758691,-0.17160787864796,0.573574068424352,0.176967718048195,-0.436206883597401,-0.0535018648884285,0.252405261951833,-0.657487754764504,-0.827135714578603,0.849573379985768,59.0,1
7519,1.23423504613468,3.0197404207034,-4.30459688479665,4.73279513041887,3.62420083055386,-1.35774566315358,1.71344498787235,-0.496358487073991,-1.28285782036322,-2.44746925511151,2.10134386504854,-4.6096283906446,1.46437762476188,-6.07933719308005,-0.339237372732577,2.58185095378146,6.73938438478335,3.04249317830411,-2.72185312222835,0.00906083639534526,-0.37906830709218,-0.704181032215427,-0.656804756348389,-1.63265295692929,1.48890144838237,0.566797273468934,-0.0100162234965625,0.146792734916988,1.0,1
7526,0.00843036489558254,4.13783683497998,-6.24069657194744,6.6757321631344,0.768307024571449,-3.35305954788994,-1.63173467271809,0.15461244822474,-2.79589246446281,-6.18789062970647,5.66439470857116,-9.85448482287037,-0.306166658250084,-10.6911962118171,-0.638498192673322,-2.04197379107768,-1.12905587703585,0.116452521226364,-1.93466573889727,0.488378221134715,0.36451420978479,-0.608057133838703,-0.539527941820093,0.128939982991813,1.48848121006868,0.50796267782385,0.735821636119662,0.513573740679437,1.0,1
7535,0.0267792264491516,4.13246389713003,-6.56059996809658,6.34855667313983,1.32966566904142,-2.51347884762413,-1.68910220031328,0.303252800547589,-3.13940905736457,-6.04546779778801,6.75462544809695,-8.94817857893317,0.702724998099873,-10.7338541032306,-1.37951985681718,-1.63896011485587,-1.74635013628103,0.776744097926754,-1.32735663549015,0.587743219006407,0.370508651493253,-0.57675247317433,-0.669605371766238,-0.759907529538618,1.60505555017462,0.540675396428899,0.737040381683977,0.496699108168337,1.0,1
7543,0.329594333318222,3.71288929524103,-5.77593510831666,6.07826550560828,1.66735901311948,-2.42016841351562,-0.812891249491333,0.133080117970748,-2.21431131204961,-5.13445447110633,4.56072010550223,-8.87374836164535,-0.797483599628474,-9.17716637009146,-0.

In [13]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import RFormula


rf = RFormula(formula="~ V1 + V2 + V3 + V4 + V5 + V6 + V7 + V8 + V9 + V10 + V11 + V12 + V13 + V14 + V15 + V16 + V17 + V18 + V19 + V20 + V21 + V22 + V23 + V24 + V25 + V26 + V27")
final_df_rf = rf.fit(df_sample).transform(df_sample)

pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(final_df_rf)

result = model.transform(final_df_rf).select("pcaFeatures")
import numpy as np
df_pca_arr = np.array(result.collect())
df_pca_no_class = sc.parallelize(df_pca_arr[:,0]).map(lambda x: x.tolist()).toDF(['V1', 'V2'])
df_pca_no_class = df_pca_no_class.withColumn("id", F.monotonically_increasing_id())
df_pca_no_class.createOrReplaceTempView('df_pca_no_class')
df_pca_no_class = spark.sql('select row_number() over (order by "id") as num, * from df_pca_no_class')
df_pca_class = df_sample.select('class')
df_pca_class = df_pca_class.withColumn("id", F.monotonically_increasing_id())
df_pca_class.createOrReplaceTempView('df_pca_class')
df_pca_class = spark.sql('select row_number() over (order by "id") as num, * from df_pca_class')

df_pca = df_pca_no_class.join(df_pca_class, "num", 'inner').drop("id")
display(df_pca)

num,V1,V2,class
1,8.683077875598103,-2.7509883906424877,1
2,0.1972612809263669,-0.5282734746561177,1
3,7.177251269775736,-3.0127958902660863,1
4,18.711151509644218,-7.740982410722202,1
5,2.3475581645592003,-2.873541741426004,1
6,14.941763047439961,-7.884547067276009,1
7,14.873980360997233,-7.918092680537963,1
8,11.40613264494057,-6.454776614523022,1
9,11.374433486109629,-6.486436525065428,1
10,13.882699306547895,-6.1057288756230985,1


In [14]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## Weighted Logistic Regression Approach

In [16]:
# Calculate the balancing ratio for the dataset
# balancing_ratio = NumberPositives / TotalRecords
balancing_ratio = float(492) / float(284807)
df_weighted = df.withColumn("weight", F.when(F.col('Class') == 0, balancing_ratio)\
                           .otherwise(1 - balancing_ratio))

## Resampling
We undersampling the normal trascation

In [18]:
df_train, df_test = df.randomSplit([0.7, 0.3], 42)

# Random Undersampling
df_fraud = df_train.filter(F.col('Class') == 1)
print df_fraud.count()
df_no_fraud_sample = df_train.filter(F.col('Class') == 0).sample(False, 0.1).limit(df_fraud.count()) #downsaple no fraud class 
print df_no_fraud_sample.count()

df_train_sample = df_fraud.union(df_no_fraud_sample)
print df_train_sample.count()

352
352
704

In [19]:
def confusionmatrix(predictions):
  if 'Class' in predictions.columns:
    tp = predictions[(predictions.Class == 1) & (predictions.prediction == 1)].count()
    tn = predictions[(predictions.Class == 0) & (predictions.prediction == 0)].count()
    fp = predictions[(predictions.Class == 0) & (predictions.prediction == 1)].count()
    fn = predictions[(predictions.Class == 1) & (predictions.prediction == 0)].count()
  else:
    tp = predictions[(predictions.label == 1) & (predictions.prediction == 1)].count()
    tn = predictions[(predictions.label == 0) & (predictions.prediction == 0)].count()
    fp = predictions[(predictions.label == 0) & (predictions.prediction == 1)].count()
    fn = predictions[(predictions.label == 1) & (predictions.prediction == 0)].count()    

  print "True Positives:", tp
  print "True Negatives:", tn
  print "False Positives:", fp
  print "False Negatives:", fn
  print "Total", predictions.count()

  r = float(tp)/(tp + fn)
  print "recall", r

  p = float(tp) / (tp + fp)
  print "precision", p

In [20]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.mllib.evaluation import BinaryClassificationMetrics

def data_prep_train(trainDF):
  
  # produce train and test dataframe objects
  featureAssembler = VectorAssembler()\
  .setInputCols(["Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12",\
                                                     "V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount"])\
    .setOutputCol("features")  

  rf = RandomForestClassifier()\
    .setLabelCol("Class")\
    .setFeaturesCol("features")\
    .setNumTrees(100)\
    .setMaxBins(32)\
    .setMaxDepth(10)

  rfpipeline = Pipeline()\
    .setStages([featureAssembler, rf])

  rfmodel=rfpipeline.fit(trainDF)
  return rfmodel

In [21]:
# Try random forest for classification
rfmodel = data_prep_train(df_train_sample)

# Make predictions
predictions = rfmodel.transform(df_test)


evaluator = MulticlassClassificationEvaluator()\
  .setLabelCol("Class")\
  .setPredictionCol("prediction")\
  .setMetricName("accuracy")
accuracy = evaluator.evaluate(predictions)

print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
confusionmatrix(predictions)

Test Accuracy = 3.00%
True Positives: 140
True Negatives: 2421
False Positives: 82914
False Negatives: 0
Total 85475
recall 1.0
precision 0.0016856502998

In [23]:
df_train, df_test = df.randomSplit([0.7, 0.3], 42)

# Random Undersampling
df_fraud = df_train.filter(F.col('Class') == 1)
print "Fraud count:", df_fraud.count()

for fraud_ratio in range (1,20):
  print "---------- Predicitions #", fraud_ratio, "----------"
  df_no_fraud_sample = df_train.filter(F.col('Class') == 0).sample(False, 0.1).limit(df_fraud.count()*fraud_ratio) #downsaple no fraud class 
  print "No fraud count:", df_no_fraud_sample.count()
  
  df_train_sample = df_fraud.union(df_no_fraud_sample)
  print "Total record count:", df_train_sample.count()
  
  # Try random forest for classification
  rfmodel = data_prep_train(df_train_sample)

  # Make predictions
  predictions = rfmodel.transform(df_test)


  evaluator = MulticlassClassificationEvaluator()\
    .setLabelCol("Class")\
    .setPredictionCol("prediction")\
    .setMetricName("accuracy")
  accuracy = evaluator.evaluate(predictions)

  print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
  confusionmatrix(predictions)

Fraud count: 352
---------- Predicitions # 1 ----------
No fraud count: 352
Total record count: 704
Test Accuracy = 3.03%
True Positives: 140
True Negatives: 2446
False Positives: 82889
False Negatives: 0
Total 85475
recall 1.0
precision 0.00168615784846
---------- Predicitions # 2 ----------
No fraud count: 704
Total record count: 1056
Test Accuracy = 6.01%
True Positives: 138
True Negatives: 5000
False Positives: 80335
False Negatives: 2
Total 85475
recall 0.985714285714
precision 0.00171486088502
---------- Predicitions # 3 ----------
No fraud count: 1056
Total record count: 1408
Test Accuracy = 5.44%
True Positives: 136
True Negatives: 4516
False Positives: 80819
False Negatives: 4
Total 85475
recall 0.971428571429
precision 0.00167994564882
---------- Predicitions # 4 ----------
No fraud count: 1408
Total record count: 1760
Test Accuracy = 12.71%
True Positives: 134
True Negatives: 10726
False Positives: 74609
False Negatives: 6
Total 85475
recall 0.957142857143
precision 0.00179281002903
---------- Predicitions # 5 ----------
No fraud count: 1760
Total record count: 2112
Test Accuracy = 14.99%
True Positives: 134
True Negatives: 12681
False Positives: 72654
False Negatives: 6
Total 85475
recall 0.957142857143
precision 0.00184096279607
---------- Predicitions # 6 ----------
No fraud count: 2112
Total record count: 2464
Test Accuracy = 23.18%
True Positives: 131
True Negatives: 19681
False Positives: 65654
False Negatives: 9
Total 85475
recall 0.935714285714
precision 0.00199133541081
---------- Predicitions # 7 ----------
No fraud count: 2464
Total record count: 2816
Test Accuracy = 15.00%
True Positives: 135
True Negatives: 12682
False Positives: 72653
False Negatives: 5
Total 85475
recall 0.964285714286
precision 0.00185470132439
---------- Predicitions # 8 ----------
No fraud count: 2816
Total record count: 3168

In [24]:
df_train, df_test = df.randomSplit([0.7, 0.3], 42)

# Random Undersampling
df_fraud = df_train.filter(F.col('Class') == 1)
no_fraud_count = df_train.filter(F.col('Class') == 0).count()
print "Fraud count:", df_fraud.count()

for fraud_ratio in range (2,5):
  print "---------- Predicitions #", fraud_ratio, "----------"
  df_no_fraud_sample = df_train.filter(F.col('Class') == 0).sample(False, 0.1).limit(no_fraud_count/fraud_ratio) #downsaple no fraud class 
  print "No fraud count:", df_no_fraud_sample.count()
  
  df_train_sample = df_fraud.union(df_no_fraud_sample)
  print "Total record count:", df_train_sample.count()
  
  # Try random forest for classification
  rfmodel = data_prep_train(df_train_sample)

  # Make predictions
  predictions = rfmodel.transform(df_test)


  evaluator = MulticlassClassificationEvaluator()\
    .setLabelCol("Class")\
    .setPredictionCol("prediction")\
    .setMetricName("accuracy")
  accuracy = evaluator.evaluate(predictions)

  print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
  confusionmatrix(predictions)

Fraud count: 352
---------- Predicitions # 2 ----------
No fraud count: 19921
Total record count: 20273
Test Accuracy = 99.93%
True Positives: 110
True Negatives: 85305
False Positives: 30
False Negatives: 30
Total 85475
recall 0.785714285714
precision 0.785714285714
---------- Predicitions # 3 ----------
No fraud count: 19747
Total record count: 20099
Test Accuracy = 99.93%
True Positives: 111
True Negatives: 85306
False Positives: 29
False Negatives: 29
Total 85475
recall 0.792857142857
precision 0.792857142857
---------- Predicitions # 4 ----------
No fraud count: 19819
Total record count: 20171
Test Accuracy = 99.94%
True Positives: 111
True Negatives: 85311
False Positives: 24
False Negatives: 29
Total 85475
recall 0.792857142857
precision 0.822222222222

In [25]:
# Continuing where the last cell stopped:
df_train, df_test = df.randomSplit([0.7, 0.3], 42)

# Random Undersampling
df_fraud = df_train.filter(F.col('Class') == 1)
print "Fraud count:", df_fraud.count()

for fraud_ratio in range (8,20):
  print "---------- Predicitions #", fraud_ratio, "----------"
  df_no_fraud_sample = df_train.filter(F.col('Class') == 0).sample(False, 0.1).limit(df_fraud.count()*fraud_ratio) #downsaple no fraud class 
  print "No fraud count:", df_no_fraud_sample.count()
  
  df_train_sample = df_fraud.union(df_no_fraud_sample)
  print "Total record count:", df_train_sample.count()
  
  # Try random forest for classification
  rfmodel = data_prep_train(df_train_sample)

  # Make predictions
  predictions = rfmodel.transform(df_test)


  evaluator = MulticlassClassificationEvaluator()\
    .setLabelCol("Class")\
    .setPredictionCol("prediction")\
    .setMetricName("accuracy")
  accuracy = evaluator.evaluate(predictions)

  print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
  confusionmatrix(predictions)

Fraud count: 352
---------- Predicitions # 8 ----------
No fraud count: 2816
Total record count: 3168
Test Accuracy = 32.12%
True Positives: 130
True Negatives: 27322
False Positives: 58013
False Negatives: 10
Total 85475
recall 0.928571428571
precision 0.00223586674234
---------- Predicitions # 9 ----------
No fraud count: 3168
Total record count: 3520
Test Accuracy = 26.68%
True Positives: 129
True Negatives: 22675
False Positives: 62660
False Negatives: 11
Total 85475
recall 0.921428571429
precision 0.00205449999204
---------- Predicitions # 10 ----------
No fraud count: 3520
Total record count: 3872
Test Accuracy = 18.10%
True Positives: 133
True Negatives: 15339
False Positives: 69996
False Negatives: 7
Total 85475
recall 0.95
precision 0.00189650501219
---------- Predicitions # 11 ----------
No fraud count: 3872
Total record count: 4224
Test Accuracy = 60.45%
True Positives: 130
True Negatives: 51539
False Positives: 33796
False Negatives: 10
Total 85475
recall 0.928571428571
precision 0.00383186936273
---------- Predicitions # 12 ----------
No fraud count: 4224
Total record count: 4576
Test Accuracy = 52.36%
True Positives: 125
True Negatives: 44633
False Positives: 40702
False Negatives: 15
Total 85475
recall 0.892857142857
precision 0.00306169936562
---------- Predicitions # 13 ----------
No fraud count: 4576
Total record count: 4928
Test Accuracy = 50.28%
True Positives: 124
True Negatives: 42852
False Positives: 42483
False Negatives: 16
Total 85475
recall 0.885714285714
precision 0.00291031990049
---------- Predicitions # 14 ----------
No fraud count: 4928
Total record count: 5280
Test Accuracy = 43.21%
True Positives: 126
True Negatives: 36807
False Positives: 48528
False Negatives: 14
Total 85475
recall 0.9
precision 0.00258971513134
---------- Predicitions # 15 ----------
No fraud count: 5280
Total record count: 5632
Test Accuracy = 39.38%
True Positives: 126
True Negatives: 33536
False Positives: 51799
False Negatives: 14
Total 85475
recall 0.9
precision 0.00242657679345
---------- Predicitions # 16 ----------
No fraud count: 5632
Total record count: 5984
Test Accuracy = 35.82%
True Positives: 127
True Negatives: 30492
False Positives: 54843
False Negatives: 13
Total 85475
recall 0.907142857143
precision 0.0023103511006
---------- Predicitions # 17 ----------
No fraud count: 5984
Total record count: 6336
Test Accuracy = 32.68%
True Positives: 128
True Negatives: 27801
False Positives: 57534
False Negatives: 12
Total 85475
recall 0.914285714286
precision 0.00221983281884
---------- Predicitions # 18 ----------
No fraud count: 6336
Total record count: 6688
Test Accuracy = 44.25%
True Positives: 129
True Negatives: 37695
False Positives: 47640
False Negatives: 11
Total 85475
recall 0.921428571429
precision 0.00270049613766
---------- Predicitions # 19 ----------
No fraud count: 6688
Total record count: 7040
Test Accuracy = 74.49%
True Positives: 127
True Negatives: 63547
False Positives: 21788
False Negatives: 13
Total 85475
recall 0.907142857143
precision 0.00579511749943

In [26]:
# Continuing where the last cell stopped:
df_train, df_test = df.randomSplit([0.7, 0.3], 42)

# Random Undersampling
df_fraud = df_train.filter(F.col('Class') == 1)
print "Fraud count:", df_fraud.count()

fraud_ratio = 20
print "---------- Predicitions #", fraud_ratio, "----------"
df_no_fraud_sample = df_train.filter(F.col('Class') == 0).sample(False, 0.1).limit(df_fraud.count()*fraud_ratio) #downsaple no fraud class 
print "No fraud count:", df_no_fraud_sample.count()

df_train_sample = df_fraud.union(df_no_fraud_sample)
print "Total record count:", df_train_sample.count()

# Try random forest for classification
rfmodel = data_prep_train(df_train_sample)

# Make predictions
predictions = rfmodel.transform(df_test)


evaluator = MulticlassClassificationEvaluator()\
  .setLabelCol("Class")\
  .setPredictionCol("prediction")\
  .setMetricName("accuracy")
accuracy = evaluator.evaluate(predictions)

print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
confusionmatrix(predictions)

Fraud count: 352
---------- Predicitions # 20 ----------
No fraud count: 7040
Total record count: 7392
Test Accuracy = 76.67%
True Positives: 126
True Negatives: 65408
False Positives: 19927
False Negatives: 14
Total 85475
recall 0.9
precision 0.00628334912482

Created graph of results in excel

In [28]:
# Keep cluster running

In [29]:
# Continuing where the last cell stopped:
df_train, df_test = df.randomSplit([0.7, 0.3], 42)

# Try random forest for classification
rfmodel = data_prep_train(df_train)

# Make predictions
predictions = rfmodel.transform(df_test)


evaluator = MulticlassClassificationEvaluator()\
  .setLabelCol("Class")\
  .setPredictionCol("prediction")\
  .setMetricName("accuracy")
accuracy = evaluator.evaluate(predictions)

print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
confusionmatrix(predictions)

Test Accuracy = 99.94%
True Positives: 104
True Negatives: 85322
False Positives: 13
False Negatives: 36
Total 85475
recall 0.742857142857
precision 0.888888888889

In [30]:
from pyspark.ml.classification import LogisticRegression

def data_prep_train(trainDF):
  
  # produce train and test dataframe objects
  featureAssembler = VectorAssembler()\
  .setInputCols(["Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12",\
                                                     "V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount"])\
    .setOutputCol("features")  

  lr = LogisticRegression(maxIter=10, regParam=0.001)\
    .setLabelCol("Class")\
    .setFeaturesCol("features")

  lrpipeline = Pipeline()\
    .setStages([featureAssembler, lr])

  lrmodel=lrpipeline.fit(trainDF)
  
  return lrmodel

In [31]:
df_train, df_test = df.randomSplit([0.7, 0.3], 42)

# Try random forest for classification
lrmodel = data_prep_train(df_train)

# Make predictions
predictions = lrmodel.transform(df_test)


evaluator = MulticlassClassificationEvaluator()\
  .setLabelCol("Class")\
  .setPredictionCol("prediction")\
  .setMetricName("accuracy")
accuracy = evaluator.evaluate(predictions)

print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
confusionmatrix(predictions)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2874208574193706> in <module> () 
 1 # Continuing where the last cell stopped: 
 ----> 2 df_train , df_test = df . randomSplit ( [ 0.7 , 0.3 ] , 42 ) 
 3 
 4 # Try random forest for classification 
 5 lrmodel = data_prep_train ( df_train ) 

 NameError : name 'df' is not defined

In [32]:
mean_Amount, sttdev_Amount = df.select(F.mean("Amount"), F.stddev("Amount")).first()
df_scaled = df.withColumn("Amount_scaled", (F.col("Amount") - mean_Amount) / sttdev_Amount)
df_scaled = df_scaled.drop('Time','Amount')
display(df_scaled)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class,Amount_scaled
-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942518,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,0.0907941719789316,-0.551599533260813,-0.617800855762348,-0.991389847235408,-0.311169353699879,1.46817697209427,-0.470400525259478,0.207971241929242,0.0257905801985591,0.403992960255733,0.251412098239705,-0.018306777944153,0.277837575558899,-0.110473910188767,0.0669280749146731,0.128539358273528,-0.189114843888824,0.133558376740387,-0.0210530534538215,0,0.24496383331667893
1.19185711131486,0.26615071205963,0.16648011335321,0.448154078460911,0.0600176492822243,-0.0823608088155687,-0.0788029833323113,0.0851016549148104,-0.255425128109186,-0.166974414004614,1.61272666105479,1.06523531137287,0.48909501589608,-0.143772296441519,0.635558093258208,0.463917041022171,-0.114804663102346,-0.183361270123994,-0.145783041325259,-0.0690831352230203,-0.225775248033138,-0.638671952771851,0.101288021253234,-0.339846475529127,0.167170404418143,0.125894532368176,-0.00898309914322813,0.0147241691924927,0,-0.34247393986494196
-1.35835406159823,-1.34016307473609,1.77320934263119,0.379779593034328,-0.503198133318193,1.80049938079263,0.791460956450422,0.247675786588991,-1.51465432260583,0.207642865216696,0.624501459424895,0.066083685268831,0.717292731410831,-0.165945922763554,2.34586494901581,-2.89008319444231,1.10996937869599,-0.121359313195888,-2.26185709530414,0.524979725224404,0.247998153469754,0.771679401917229,0.909412262347719,-0.689280956490685,-0.327641833735251,-0.139096571514147,-0.0553527940384261,-0.0597518405929204,0,1.1606838875569188
-0.966271711572087,-0.185226008082898,1.79299333957872,-0.863291275036453,-0.0103088796030823,1.24720316752486,0.23760893977178,0.377435874652262,-1.38702406270197,-0.0549519224713749,-0.226487263835401,0.178228225877303,0.507756869957169,-0.28792374549456,-0.631418117709045,-1.0596472454325,-0.684092786345479,1.96577500349538,-1.2326219700892,-0.208037781160366,-0.108300452035545,0.00527359678253453,-0.190320518742841,-1.17557533186321,0.647376034602038,-0.221928844458407,0.0627228487293033,0.0614576285006353,0,0.1405340052658796
-1.15823309349523,0.877736754848451,1.548717846511,0.403033933955121,-0.407193377311653,0.0959214624684256,0.592940745385545,-0.270532677192282,0.817739308235294,0.753074431976354,-0.822842877946363,0.53819555014995,1.3458515932154,-1.11966983471731,0.175121130008994,-0.451449182813529,-0.237033239362776,-0.0381947870352842,0.803486924960175,0.408542360392758,-0.00943069713232919,0.79827849458971,-0.137458079619063,0.141266983824769,-0.206009587619756,0.502292224181569,0.219422229513348,0.215153147499206,0,-0.07340321138795917
-0.425965884412454,0.960523044882985,1.14110934232219,-0.168252079760302,0.42098688077219,-0.0297275516639742,0.476200948720027,0.260314333074874,-0.56867137571251,-0.371407196834471,1.34126198001957,0.359893837038039,-0.358090652573631,-0.137133700217612,0.517616806555742,0.401725895589603,-0.0581328233640131,0.0686531494425432,-0.0331937877876282,0.0849676720682049,-0.208253514656728,-0.559824796253248,-0.0263976679795373,-0.371426583174346,-0.232793816737034,0.105914779097957,0.253844224739337,0.0810802569229443,0,-0.3385558222734495
1.22965763450793,0.141003507049326,0.0453707735899449,1.20261273673594,0.191880988597645,0.272708122899098,-0.00515900288250983,0.0812129398830894,0.464959994783886,-0.0992543211289237,-1.41690724314928,-0.153825826253651,-0.75106271556262,0.16737196252175,0.0501435942254188,-0.443586797916727,0.00282051247234708,-0.61198733994012,-0.0455750446637976,-0.21963255278686,-0.167716265815783,-0.270709726172363,-0.154103786809305,-0.780055415004671,0.75013693580659,-0.257236845917139,0.0345074297438413,0.00516776890624916,0,-0.3332783577624597
-0.644269442348146,1.41796354547385,1.0743803763556,-0.492199018495015,0.948934094764157,0.428118462833089,1

In [33]:
def data_prep_train_scaled(trainDF):
  
  # produce train and test dataframe objects
  featureAssembler = VectorAssembler()\
  .setInputCols(["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12",\
                                                     "V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount_scaled"])\
    .setOutputCol("features")  

  rf = RandomForestClassifier()\
    .setLabelCol("Class")\
    .setFeaturesCol("features")\
    .setNumTrees(100)\
    .setMaxBins(32)\
    .setMaxDepth(10)

  rfpipeline = Pipeline()\
    .setStages([featureAssembler, rf])

  rfmodel=rfpipeline.fit(trainDF)
  return rfmodel

In [34]:
import pandas as pd
from imblearn.over_sampling import SMOTE,RandomOverSampler
from sklearn.model_selection import train_test_split
from collections import Counter
df_pd = df.toPandas()

#df_train, df_test = df.randomSplit([0.7, 0.3], 42)
#df_train_pd = df_train.toPandas()

X = df_pd.iloc[:, df_pd.columns != 'Class']
Y = df_pd.iloc[:, df_pd.columns == 'Class']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

sm = SMOTE(random_state=12, ratio = 'auto', kind = 'regular')

x_train_res, y_train_res = sm.fit_sample(x_train, y_train)
print 'Resampled dataset shape {}'.format(Counter(y_train_res))

Resampled dataset shape Counter({0: 199008, 1: 199008})

In [35]:
xcol = df.columns
ycol = [xcol.pop()]
df_pd_res_x = pd.DataFrame(x_train_res, columns=xcol)
df_pd_res_y = pd.DataFrame(y_train_res, columns=ycol)
df_pd_res = pd.concat([df_pd_res_x,df_pd_res_y], axis=1)
df_pd_res.head()

Out[ 105 ]: 
 Time V1 V2 V3 V4 V5 V6 \
0 2105.0 -2.289565 -0.480260 0.818685 -1.706423 0.822102 -1.660326 
1 153702.0 -0.313717 -4.064342 -3.398445 0.704011 0.101662 1.529848 
2 97283.0 -1.809763 -0.567439 2.265186 -0.960318 -1.212537 1.516493 
3 68628.0 1.192319 0.178575 0.141491 0.459628 -0.049959 -0.112122 
4 11951.0 -0.963451 0.700311 1.097333 -1.547626 0.669966 0.513533 

 V7 V8 V9 ... V21 V22 V23 \
0 0.944047 -0.541765 1.323156 ... -0.210837 0.914737 0.867888 
1 1.551670 -0.036774 0.015829 ... 0.853186 -0.091941 -0.936215 
2 -1.417176 0.903421 1.961027 ... -0.509915 -0.424978 -0.268621 
3 -0.163883 0.155740 -0.067566 ... -0.240464 -0.739862 0.116799 
4 0.333683 0.270900 1.381880 ... -0.279519 -0.470181 -0.124037 

 V24 V25 V26 V27 V28 Amount Class 
0 0.422969 0.310584 -0.781488 0.392241 -0.147757 1.00 0 
1 -0.833081 -0.498728 0.651183 -0.290331 0.110360 1194.28 0 
2 0.010121 0.466862 0.835540 -0.062385 0.088079 75.00 0 
3 -0.373837 0.125470 0.130126 -0.016956 0.011937 1.98 0 
4 -1.388839 -0.237453 0.785347 0.349708 0.216207 37.31 0 

[5 rows x 31 columns]

In [36]:
df_pd_x_test = pd.DataFrame(x_test, columns=xcol)
df_pd_y_test = pd.DataFrame(y_test, columns=ycol)
df_pd_test = pd.concat([df_pd_x_test,df_pd_y_test], axis=1)
df_pd_test.head()

Out[ 108 ]: 
 Time V1 V2 V3 V4 V5 V6 \
43428 41505 -16.526507 8.584972 -18.649853 9.505594 -13.793819 -2.832404 
49906 44261 0.339812 -2.743745 -0.134070 -1.385729 -1.451413 1.015887 
29474 35484 1.399590 -0.590701 0.168619 -1.029950 -0.539806 0.040444 
276481 167123 -0.432071 1.647895 -1.669361 -0.349504 0.785785 -0.630647 
278846 168473 2.014160 -0.137394 -1.015839 0.327269 -0.182179 -0.956571 

 V7 V8 V9 ... V21 V22 V23 \
43428 -16.701694 7.517344 -8.507059 ... 1.190739 -1.127670 -2.358579 
49906 -0.524379 0.224060 0.899746 ... -0.213436 -0.942525 -0.526819 
29474 -0.712567 0.002299 -0.971747 ... 0.102398 0.168269 -0.166639 
276481 0.276990 0.586025 -0.484715 ... 0.358932 0.873663 -0.178642 
278846 0.043241 -0.160746 0.363241 ... -0.238644 -0.616400 0.347045 

 V24 V25 V26 V27 V28 Amount Class 
43428 0.673461 -1.413700 -0.462762 -2.018575 -1.042804 364.19 1 
49906 -1.156992 0.311211 -0.746647 0.040996 0.102038 520.12 0 
29474 -0.810250 0.505083 -0.232340 0.011409 0.004634 31.00 0 
276481 -0.017171 -0.207392 -0.157756 -0.237386 0.001934 1.50 0 
278846 0.061561 -0.360196 0.174730 -0.078043 -0.070571 0.89 0 

[5 rows x 31 columns]

In [37]:
df_train_res = spark.createDataFrame(df_pd_res)
df_test = spark.createDataFrame(df_pd_test)

In [38]:
# Try random forest for classification
rfmodel = data_prep_train(df_train_res)

# Make predictions
predictions = rfmodel.transform(df_test)

evaluator = MulticlassClassificationEvaluator()\
  .setLabelCol("Class")\
  .setPredictionCol("prediction")\
  .setMetricName("accuracy")
accuracy = evaluator.evaluate(predictions)

print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
confusionmatrix(predictions)

Test Accuracy = 99.86%
True Positives: 123
True Negatives: 85199
False Positives: 108
False Negatives: 13
Total 85443
recall 0.904411764706
precision 0.532467532468

In [39]:
mean_Amount, sttdev_Amount = df.select(F.mean("Amount"), F.stddev("Amount")).first()
df_scaled = df.withColumn("Amount_scaled", (F.col("Amount") - mean_Amount) / sttdev_Amount)
df_scaled = df_scaled.drop('Time','Amount')

df_pd = df_scaled.toPandas()

X = df_pd.iloc[:, df_pd.columns != 'Class']
Y = df_pd.iloc[:, df_pd.columns == 'Class']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

sm = SMOTE(random_state=12, ratio = 'auto', kind = 'regular')

x_train_res, y_train_res = sm.fit_sample(x_train, y_train)

xcol = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount_scaled"]
ycol = ["Class"]
df_pd_res_x = pd.DataFrame(x_train_res, columns=xcol)
df_pd_res_y = pd.DataFrame(y_train_res, columns=ycol)
df_pd_res = pd.concat([df_pd_res_x,df_pd_res_y], axis=1)

df_pd_x_test = pd.DataFrame(x_test, columns=xcol)
df_pd_y_test = pd.DataFrame(y_test, columns=ycol)
df_pd_test = pd.concat([df_pd_x_test,df_pd_y_test], axis=1)

df_train_res = spark.createDataFrame(df_pd_res)
df_test = spark.createDataFrame(df_pd_test)

# Try random forest for classification
rfmodel = data_prep_train_scaled(df_train_res)

# Make predictions
predictions = rfmodel.transform(df_test)

evaluator = MulticlassClassificationEvaluator()\
  .setLabelCol("Class")\
  .setPredictionCol("prediction")\
  .setMetricName("accuracy")
accuracy = evaluator.evaluate(predictions)

print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
confusionmatrix(predictions)

Test Accuracy = 99.75%
True Positives: 122
True Negatives: 85105
False Positives: 202
False Negatives: 14
Total 85443
recall 0.897058823529
precision 0.376543209877

In [40]:
# DOES NOT WORK IN SPARK
from imblearn.over_sampling import SMOTE,RandomOverSampler

df_train, df_test = df.randomSplit([0.7, 0.3], 42)

xcol = df_train.columns
ycol = xcol.pop()

df_train_x = df_train.select(xcol) # Feature columns
df_train_y = df_train.select(ycol) # Class column


df_train_x_array =  np.array(df_train_x.select(df_train_x.columns).collect())
df_train_y_array =  np.array(df_train_y.select(df_train_y.columns).collect())

sm = SMOTE(random_state=12, ratio = 'auto', kind = 'regular')
X_resampled, y_resampled = sm.fit_sample(df_train_x_array, df_train_y_array)

In [41]:
df_X_resampled = sc.parallelize(X_resampled).toDF(df_train_x.columns)
#X_resampled

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-2575811041299719> in <module> () 
 ----> 1 df_X_resampled = sc . parallelize ( X_resampled ) . toDF ( df_train_x . columns ) 
 2 #X_resampled 

 /databricks/spark/python/pyspark/sql/session.py in toDF (self, schema, sampleRatio) 
 58 [ Row ( name = u'Alice' , age = 1 ) ] 
 59 """
 ---> 60 return sparkSession . createDataFrame ( self , schema , sampleRatio ) 
 61 
 62 RDD . toDF = toDF 

 /databricks/spark/python/pyspark/sql/session.py in createDataFrame (self, data, schema, samplingRatio, verifySchema) 
 725 else : 
 726 if isinstance ( data , RDD ) : 
 --> 727 rdd , schema = self . _createFromRDD ( data . map ( prepare ) , schema , samplingRatio ) 
 728 else : 
 729 rdd , schema = self . _createFromLocal ( map ( prepare , data ) , schema ) 

 /databricks/spark/python/pyspark/sql/session.py in _createFromRDD (self, rdd, schema, samplingRatio) 
 384 """
 385 if schema is None or isinstance ( schema , ( list , tuple ) ) : 
 --> 386 struct = self . _inferSchema ( rdd , samplingRatio , names = schema ) 
 387 converter = _create_converter ( struct ) 
 388 rdd = rdd . map ( converter ) 

 /databricks/spark/python/pyspark/sql/session.py in _inferSchema (self, rdd, samplingRatio, names) 
 356 """
 357 first = rdd . first ( ) 
 --> 358 if not first : 
 359 raise ValueError("The first row in RDD is empty, "
 360 "can not infer schema")

 ValueError : The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

### Model selection (a.k.a. hyperparameter tuning)
An important task in ML is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LogisticRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately.
https://spark.apache.org/docs/latest/ml-tuning.html

In [43]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [44]:
# produce train and test dataframe objects
featureAssembler = VectorAssembler()\
 .setInputCols(["Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount"])\
  .setOutputCol("features")  

rf_cv = RandomForestClassifier()\
  .setLabelCol("label")\
  .setFeaturesCol("features")\
  .setNumTrees(10)\
  .setMaxBins(50)\
  .setMaxDepth(20)\

pipeline_cv = Pipeline(stages=[featureAssembler,rf_cv])

trainDF_cv = trainDF.withColumnRenamed('Class', 'label')
testDF_cv = testDF.withColumnRenamed('Class', 'label')

paramGrid = ParamGridBuilder() \
    .addGrid(rf_cv.setNumTrees, [75, 100, 125]) \
    .addGrid(rf_cv.setMaxBins, [20, 32, 45])\
    .addGrid(rf_cv.setMaxDepth, [8, 10, 15])\
    .build()

crossval = CrossValidator(estimator=pipeline_cv,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=10)

cvModel = crossval.fit(trainDF_cv)

In [45]:
prediction = cvModel.transform(testDF_cv)

evaluator = MulticlassClassificationEvaluator()\
  .setLabelCol("label")\
  .setPredictionCol("prediction")\
  .setMetricName("accuracy")
accuracy = evaluator.evaluate(prediction)

print "Accuracy = %5.2f%%" % ((accuracy)*100)

confusionmatrix(prediction)

Accuracy = 99.29%
True Positives: 152
True Negatives: 128
False Positives: 1
False Negatives: 1
Total 284807
recall 0.993464052288
precision 0.993464052288

In [46]:
#make prediction on the unsee imbalanced dataset 
im_prediction = cvModel.transform(im_test)
confusionmatrix(im_prediction)

True Positives: 50
True Negatives: 668
False Positives: 27785
False Negatives: 0
Total 284807
recall 1.0
precision 0.00179629962278

try slightly imbalance data for trining

In [48]:
df_weighted_train, im_test = df_weighted.randomSplit([0.9, 0.1], 42)

# Random Undersampling
df_fraud = df_weighted_train.filter(F.col('Class') == 1)
print df_fraud.count()
df_no_fraud_sample = df_weighted_train.filter(F.col('Class') == 0).sample(False, 0.1).limit(3*df_fraud.count()) #downsaple no fraud class 
print df_no_fraud_sample.count()

df_weighted_sample = df_fraud.union(df_no_fraud_sample)
print df_weighted_sample.count()
# Random Oversampling


# First split the data without sampling for comparison
splits = df_weighted_sample.randomSplit([0.7, 0.3], 42) # 70/30 split for training/testing
(trainDF, testDF) = (splits[0], splits[1])

442
1326
1768

In [49]:
# Try random forest for classification
rfmodel = data_prep_train(trainDF)

# Make predictions
predictions = rfmodel.transform(testDF)

evaluator = MulticlassClassificationEvaluator()\
  .setLabelCol("Class")\
  .setPredictionCol("prediction")\
  .setMetricName("accuracy")
accuracy = evaluator.evaluate(predictions)

print "Test Accuracy = %5.2f%%" % ((accuracy)*100)
confusionmatrix(predictions)

Test Accuracy = 99.63%
True Positives: 151
True Negatives: 393
False Positives: 0
False Negatives: 2
Total 284807
recall 0.986928104575
precision 1.0

In [50]:
im_prediction = rfmodel.transform(im_test)
confusionmatrix(im_prediction)

True Positives: 49
True Negatives: 2722
False Positives: 25731
False Negatives: 1
Total 284807
recall 0.98
precision 0.00190069821567